In [1]:

from sklearn.datasets import make_circles
X, y = make_circles(1000,noise=0.03, random_state=42)

import torch
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)
X[:5], y[:5]

(tensor([[ 0.7542,  0.2315],
         [-0.7562,  0.1533],
         [-0.8154,  0.1733],
         [-0.3937,  0.6929],
         [ 0.4422, -0.8967]]),
 tensor([1., 1., 1., 1., 0.]))

In [2]:
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = 100 * correct /  len(y_pred)
    return acc

In [3]:
import requests
from pathlib import Path 

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

from helper_functions import plot_predictions, plot_decision_boundary


In [4]:
from torch import nn
model = nn.Sequential(
    nn.Linear(2,5),
    nn.ReLU(),
    nn.Linear(5,5),
    nn.ReLU(),
    nn.Linear(5,1),

)

loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
print(model)

Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=5, bias=True)
  (3): ReLU()
  (4): Linear(in_features=5, out_features=1, bias=True)
)


In [5]:
device='cpu'

from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:

torch.manual_seed(42)
epochs = 1500

for epoch in range(epochs):
    train_logits = model(X_train).squeeze()
    train_pred = torch.round(torch.sigmoid(train_logits))
    loss = loss_fn(train_logits, y_train)# BCEWithLogitsLoss calculates loss using logits
    acc = accuracy_fn(y_train, train_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    with torch.inference_mode():
        test_logits = model(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))
        test_loss = loss_fn(test_pred, y_test)
        test_acc = accuracy_fn(y_test, test_pred)

        if epoch %100 == 0:
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc:.2f}%") 



Epoch: 0 | Loss: 0.70278, Accuracy: 50.00% | Test Loss: 0.81326, Test Accuracy: 50.00%
Epoch: 100 | Loss: 0.69209, Accuracy: 50.88% | Test Loss: 0.73879, Test Accuracy: 50.00%
Epoch: 200 | Loss: 0.68992, Accuracy: 53.00% | Test Loss: 0.72848, Test Accuracy: 50.50%
Epoch: 300 | Loss: 0.68756, Accuracy: 54.88% | Test Loss: 0.71968, Test Accuracy: 52.50%
Epoch: 400 | Loss: 0.68459, Accuracy: 55.88% | Test Loss: 0.72158, Test Accuracy: 52.00%
Epoch: 500 | Loss: 0.68071, Accuracy: 56.75% | Test Loss: 0.71399, Test Accuracy: 54.00%
Epoch: 600 | Loss: 0.67549, Accuracy: 57.25% | Test Loss: 0.71709, Test Accuracy: 53.50%
Epoch: 700 | Loss: 0.66833, Accuracy: 58.62% | Test Loss: 0.72139, Test Accuracy: 53.00%
Epoch: 800 | Loss: 0.65798, Accuracy: 59.12% | Test Loss: 0.71689, Test Accuracy: 54.50%
Epoch: 900 | Loss: 0.64127, Accuracy: 61.75% | Test Loss: 0.71189, Test Accuracy: 55.50%
Epoch: 1000 | Loss: 0.60924, Accuracy: 72.00% | Test Loss: 0.64949, Test Accuracy: 67.50%
Epoch: 1100 | Loss: 0.